## Problem Description

Given the advent of shared workspaces, I believe it is a good idea to research the perfect location of setting up a shared workspace in Philadelphia. Given that Philadelphia is home to a lot on educational institutions and students are always coming up with great ideas that can be the next big thing. One of the examples is GoPuff which recently received a major investment from Softbank. 

Firstly, A perfect shared workspace would be a place which is near the educational institutions allowing student contributors to come and go easily and even at odd hours, given that the startups and small team companies have a varied working schedule for hours.

Secondly the workspace should be adequetly be accesible by public transport systems, so bus or train stations will also be measured in.

Another factor to measure is the availability of restaurants and other food joints and also making sure that they are open at the odd hours in case the companies want to work in late and order food.

The plan is to start with these basic parameters and then see if we can measure in other factors to enhance the search for a perfect shared workspace in Philadelphia. 

Let the search begin!

## Data

The data we will be using will be Foursquare location data to track proximity to institutions, public transport hotspots and restaurants / eateries etc. for the city of Philadelphia. Also, We will be making use of the following packages that are imported in the cell below. These are basically used for getting location data, address information, clustering library and matplotlib and folium for plotting modules and map rendering respectively.

In [ ]:
import pandas as pd
import numpy as np
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library



Solving environment: done

# All requested packages already installed.

Solving environment: | 

In [ ]:
pip install pgeocode

## Geolocator to get location Coordinates for Philadelphia

In [40]:

address = 'Philadelphia, US'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Philadelphia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Philadelphia are 39.9527237, -75.1635262.


## Pgeocode to get the latitudes and longitudes for various neighborhoods for Philadelphia and also ease of calculating distances between lat,long duos

In [41]:
import pgeocode
nomi = pgeocode.Nominatim('us')
zips = ["19102","19103","19104","19106","19107","19109","19111","19112","19114","19115","19116","19118","19119","19120","19121","19122","19123","19124","19125","19126","19127","19128","19129","19130","19131","19132","19133","19134","19135","19136","19137","19138","19139","19140","19141","19142","19143","19144","19145","19146","19147","19148","19149","19150","19151","19152","19153","19154"]
phillyData = nomi.query_postal_code(zips).head()
phillyData

,postal_code,country code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,19102,US,Philadelphia,Pennsylvania,PA,Philadelphia,101.0,NaN,NaN,39.9489,-75.1661,4.0
1,19103,US,Philadelphia,Pennsylvania,PA,Philadelphia,101.0,NaN,NaN,39.9513,-75.1741,4.0
2,19104,US,Philadelphia,Pennsylvania,PA,Philadelphia,101.0,NaN,NaN,39.9597,-75.2024,4.0
3,19106,US,Philadelphia,Pennsylvania,PA,Philadelphia,101.0,NaN,NaN,39.9474,-75.1473,4.0
4,19107,US,Philadelphia,Pennsylvania,PA,Philadelphia,101.0,NaN,NaN,39.9487,-75.1593,4.0


## Reverse locator to get Neighborhood names, since Pgeocode is missing those

In [42]:
locator = Nominatim(user_agent="myGeocoder")
coordinates = "39.9597, -75.2024"
location = locator.reverse(coordinates)
location.raw

{'place_id': 186534781,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 452087322,
 'lat': '39.959441749999996',
 'lon': '-75.20236029126505',
 'display_name': 'Getty Gas, North 40th Street, Powelton Village, Philadelphia, Philadelphia County, Pennsylvania, 19104, United States of America',
 'address': {'amenity': 'Getty Gas',
  'road': 'North 40th Street',
  'neighbourhood': 'Powelton Village',
  'city': 'Philadelphia',
  'county': 'Philadelphia County',
  'state': 'Pennsylvania',
  'postcode': '19104',
  'country': 'United States of America',
  'country_code': 'us'},
 'boundingbox': ['39.9592531', '39.9596304', '-75.2026155', '-75.2020982']}

## And Finally Foursquare API data to get description and location of venues

In [43]:
CLIENT_ID = '10UP2LT2HSBHAECSOXEODQS4RMO2DYHNMJTRWFBR2Z2REVLK' # your Foursquare ID
CLIENT_SECRET = 'OVV3FEQZE2Z4TEOJ2KCHCD24PJKXGXRAU15ZIIHE4TBIJYZL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

philly_venues = getNearbyVenues(names=phillyData['place_name'],
                                   latitudes=phillyData['latitude'],
                                   longitudes=phillyData['longitude']
                                  )

philly_venues

Your credentails:
CLIENT_ID: 10UP2LT2HSBHAECSOXEODQS4RMO2DYHNMJTRWFBR2Z2REVLK
CLIENT_SECRET:OVV3FEQZE2Z4TEOJ2KCHCD24PJKXGXRAU15ZIIHE4TBIJYZL


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Philadelphia,39.9489,-75.1661,The Sporting Club at the Bellevue,39.948691,-75.164601,Gym
1,Philadelphia,39.9489,-75.1661,Elixr Coffee Roasters,39.949243,-75.166716,Coffee Shop
2,Philadelphia,39.9489,-75.1661,Oyster House,39.950415,-75.166533,Seafood Restaurant
3,Philadelphia,39.9489,-75.1661,Mission Taqueria,39.950389,-75.166504,Taco Place
4,Philadelphia,39.9489,-75.1661,Rescue Spa,39.950133,-75.167464,Spa
...,...,...,...,...,...,...,...
420,Philadelphia,39.9487,-75.1593,Termini Brothers Bakery,39.953060,-75.159560,Bakery
421,Philadelphia,39.9487,-75.1593,The Capital Grille,39.950716,-75.163942,American Restaurant
422,Philadelphia,39.9487,-75.1593,Knead Bagels,39.948051,-75.153640,Bagel Shop
423,Philadelphia,39.9487,-75.1593,Winkel,39.947793,-75.160339,Breakfast Spot
